# Bi-Directional RNN

In [1]:
import os
import sys
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms

print(f'torch version is : {torch.__version__}')

torch version is : 1.11.0+cpu


In [6]:
# Hyper-parameters
epochs = 2
num_layers = 2
input_size = 28
num_classes = 10
batch_size = 100
hidden_size = 128
sequence_length = 28
learning_rate = 0.003
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from torchvision.datasets import MNIST

train_dataset = MNIST(
  root='../basics/mnist',
  train=True, 
  transform=transforms.ToTensor(), 
  download=False  
)

test_dataset = MNIST(
  root='../basics/mnist',
  train=False, 
  transform=transforms.ToTensor(), 
  download=False  
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../basics/mnist\MNIST\raw\train-images-idx3-ubyte.gz to ../basics/mnist\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../basics/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ../basics/mnist\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../basics/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ../basics/mnist\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../basics/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../basics/mnist\MNIST\raw



In [4]:
# Define dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [7]:
# Define BiRNN
class BiRNN(nn.Module):
    # Define class parameters
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        # Set parameters
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Define the layers 
        self.lstm = nn.LSTM(
            input_size, 
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [8]:
# define the loss and optimizer
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        l = loss(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, l.item()))

AttributeError: 'CrossEntropyLoss' object has no attribute 'item'